# Ontologies

* An ontology defines the types of vector tools and classifications available for a project
* Currently the only way to create an ontology object is by setting up a project with one.
* This notebook will cover client side tools for working with ontologies
* Project setup is explained here: https://github.com/Labelbox/labelbox-python/blob/develop/examples/project_configuration/project_setup.ipynb


In [1]:
!pip install labelbox

In [2]:
from labelbox.schema.ontology import OntologyBuilder, Tool, Classification, Option
from labelbox import Client
from getpass import getpass
import os

In [3]:
# If you don't want to give google access to drive you can skip this cell
# and manually set `API_KEY` below.

COLAB = "google.colab" in str(get_ipython())
if COLAB:
    !pip install colab-env -qU
    from colab_env import envvar_handler
    envvar_handler.envload()

API_KEY = os.environ.get("LABELBOX_API_KEY")
if not os.environ.get("LABELBOX_API_KEY"):
    API_KEY = getpass("Please enter your labelbox api key")
    if COLAB:
        envvar_handler.add_env("LABELBOX_API_KEY", API_KEY)

* Set the following cell with your data to run this notebook

In [4]:
# Make sure this project has been fully setup (see project examples/project_configuration/project_setup.ipynb)
# You also can just setup a project in the web ui
PROJECT_ID = "ckm4xyfncfgja0760vpfdxoro"
# Only update this if you have an on-prem deployment
ENDPOINT = "https://api.labelbox.com/graphql"

In [5]:
client = Client(api_key=API_KEY, endpoint=ENDPOINT)

### Create
* Note that this a client side tool only. See project setup to attach this to a project

In [6]:
ontology_builder = OntologyBuilder(tools=[
    Tool(tool=Tool.Type.BBOX, name="dog"),
    Tool(tool=Tool.Type.SEGMENTATION, name="cat"),
])
# Note that color is selected automatically
# The ids will be assigned when this ontology is created in labelbox
print(ontology_builder)

{
  "tools": [
    {
      "tool": "rectangle",
      "name": "dog",
      "required": false,
      "color": "#ff0000",
      "classifications": [],
      "schemaNodeId": null,
      "featureSchemaId": null
    },
    {
      "tool": "superpixel",
      "name": "cat",
      "required": false,
      "color": "#00ffff",
      "classifications": [],
      "schemaNodeId": null,
      "featureSchemaId": null
    }
  ],
  "classifications": []
}


In [7]:
# Adding to the tool over time
ontology_builder = OntologyBuilder()
tool_dog = Tool(tool=Tool.Type.BBOX, name="dog")
tool_cat = Tool(tool=Tool.Type.SEGMENTATION, name="cat")
ontology_builder.add_tool(tool_dog)
ontology_builder.add_tool(tool_cat)
# Exactly the same as the using the init
print(ontology_builder)

{
  "tools": [
    {
      "tool": "rectangle",
      "name": "dog",
      "required": false,
      "color": "#ff0000",
      "classifications": [],
      "schemaNodeId": null,
      "featureSchemaId": null
    },
    {
      "tool": "superpixel",
      "name": "cat",
      "required": false,
      "color": "#00ffff",
      "classifications": [],
      "schemaNodeId": null,
      "featureSchemaId": null
    }
  ],
  "classifications": []
}


In [8]:
# Classifications are supported too (Both for top level and as subclassifications)
ontology_builder = OntologyBuilder(
    tools=[
        Tool(tool=Tool.Type.BBOX, name="dog"),
        Tool(tool=Tool.Type.SEGMENTATION,
             name="cat",
             classifications=[
                 Classification(class_type=Classification.Type.TEXT,
                                instructions="name")
             ])
    ],
    classifications=[
        Classification(class_type=Classification.Type.RADIO,
                       instructions="image_quality",
                       options=[Option(value="clear"),
                                Option(value="blurry")])
    ])
print(ontology_builder)

{
  "tools": [
    {
      "tool": "rectangle",
      "name": "dog",
      "required": false,
      "color": "#ff0000",
      "classifications": [],
      "schemaNodeId": null,
      "featureSchemaId": null
    },
    {
      "tool": "superpixel",
      "name": "cat",
      "required": false,
      "color": "#00ffff",
      "classifications": [
        {
          "type": "text",
          "instructions": "name",
          "name": "name",
          "required": false,
          "options": [],
          "schemaNodeId": null,
          "featureSchemaId": null
        }
      ],
      "schemaNodeId": null,
      "featureSchemaId": null
    }
  ],
  "classifications": [
    {
      "type": "radio",
      "instructions": "image_quality",
      "name": "image_quality",
      "required": false,
      "options": [
        {
          "schemaNodeId": null,
          "featureSchemaId": null,
          "label": "clear",
          "value": "clear",
          "options": []
        },
        {
     

In [9]:
# Note that all Tool objects are constructed the same way:
bbox_tool = Tool(tool=Tool.Type.BBOX, name="dog_box")
poly_tool = Tool(tool=Tool.Type.POLYGON, name="dog_poly")
seg_tool = Tool(tool=Tool.Type.SEGMENTATION, name="dog_seg")
point_tool = Tool(tool=Tool.Type.POINT, name="dog_center")
line_tool = Tool(tool=Tool.Type.LINE, name="dog_orientation")
ner_tool = Tool(tool=Tool.Type.NER, name="dog_reference")
# All with optional colors,  and subclass

In [10]:
# Classifications are all constructed the same way except text which doesn't require options
# Classifications can belong to the full data_row (image,text, or video classification)
#     Or they can be subclasses to a tool (ie dog bounding box, with a breed classification)

text_classification = Classification(class_type=Classification.Type.TEXT,
                                     instructions="dog_name")
radio_classification = Classification(class_type=Classification.Type.CHECKLIST,
                                      instructions="dog_breed",
                                      options=[Option("poodle")])
dropdown_classification = Classification(
    class_type=Classification.Type.DROPDOWN,
    instructions="dog_features",
    options=[Option("short"), Option("fluffy")])
checklist_classification = Classification(
    class_type=Classification.Type.CHECKLIST,
    instructions="background",
    options=[Option("at_park"), Option("has_leash")])

### Read


In [11]:
# We can fetch ontologies by its relationship to a project
project = client.get_project(PROJECT_ID)
ontology = project.ontology()

In [12]:
# This ontology is an object that represents an ontology that exists in labelbox
#  (unlike the ontology builder that only exists in your local python version)
# To construct a local ontology from a ontology db object you can do:
OntologyBuilder.from_dict(ontology.normalized)

OntologyBuilder(tools=[Tool(tool=<Type.BBOX: 'rectangle'>, name='animal', required=False, color='#1CE6FF', classifications=[], schema_id='ckm3se1a5010x0y6r3hpsb0wc', feature_schema_id='ckm3se1a5010y0y6rcdc0bvvl'), Tool(tool=<Type.BBOX: 'rectangle'>, name='human', required=False, color='#FF34FF', classifications=[], schema_id='ckmbdol5s1ybj0y969w8z6fcs', feature_schema_id='ckmbdol5s1ybk0y96fje86ktu')], classifications=[Classification(class_type=<Type.CHECKLIST: 'checklist'>, instructions='image', required=False, options=[Option(value='difficult', schema_id='ckm3se1aw010z0y6rdgaof1fu', feature_schema_id='ckm3se1aw01100y6rf8b59u51', options=[]), Option(value='contains_human', schema_id='ckm3se1aw01110y6rfx3benbx', feature_schema_id='ckm3se1aw01120y6r7zkcd93v', options=[])], schema_id='ckm3se1a4010v0y6rfe3b9s8t', feature_schema_id='ckm3se1a4010w0y6rd579gyl3')])

In [13]:
# You can even do
OntologyBuilder.from_project(project)

OntologyBuilder(tools=[Tool(tool=<Type.BBOX: 'rectangle'>, name='animal', required=False, color='#1CE6FF', classifications=[], schema_id='ckm3se1a5010x0y6r3hpsb0wc', feature_schema_id='ckm3se1a5010y0y6rcdc0bvvl'), Tool(tool=<Type.BBOX: 'rectangle'>, name='human', required=False, color='#FF34FF', classifications=[], schema_id='ckmbdol5s1ybj0y969w8z6fcs', feature_schema_id='ckmbdol5s1ybk0y96fje86ktu')], classifications=[Classification(class_type=<Type.CHECKLIST: 'checklist'>, instructions='image', required=False, options=[Option(value='difficult', schema_id='ckm3se1aw010z0y6rdgaof1fu', feature_schema_id='ckm3se1aw01100y6rf8b59u51', options=[]), Option(value='contains_human', schema_id='ckm3se1aw01110y6rfx3benbx', feature_schema_id='ckm3se1aw01120y6r7zkcd93v', options=[])], schema_id='ckm3se1a4010v0y6rfe3b9s8t', feature_schema_id='ckm3se1a4010w0y6rd579gyl3')])

In [14]:
# Note that this includes all the extra information about the ontology that was created for you
# (Such as Ids). These are useful for uploading

### Update and Delete
- At this time, these options are not supported from the SDK.
- Updating an ontology is dangerous and could cause labels to be hidden. 
    - Use caution when doing so